# Logical error rate of BB code (with different cluster_node_limit for MWPF decoder)

In [1]:
# This cell is tagged `parameters` and will be override by `papermill`
json_filename: str = "bb72.json"  # where to save the result, must be provided
code: str = "bb(n=72,k=12,d=6)"
noise: str = "depolarize(p=0.01)"

slurm_maximum_jobs: int = 500
slurm_cores_per_node: int = 10  # (slurm_maximum_jobs // slurm_cores_per_node) should not exceed 200
slurm_mem_per_job: int = 4  # 4GB per job
slurm_walltime: str = "11:00:00"  # 11 hours (adaptively shutdown if no more jobs)
slurm_partition: str = "scavenge"

local_maximum_jobs: int = 10

min_shots: int = 10_000_000  # p_L between 6.7e-5 ~ 3.9e-5

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from qec_lego_bench.hpc.monte_carlo import MonteCarloJob, LogicalErrorResult, MonteCarloJobExecutor
from qec_lego_bench.hpc.submitter.min_shots_submitter import MinShotsSubmitter
from qec_lego_bench.hpc.submitter.precision_submitter import PrecisionSubmitter
from qec_lego_bench.hpc.plotter.job_progress_plotter import JobProgressPlotter
from qec_lego_bench.hpc.plotter.logical_error_rate_plotter import LogicalErrorRatePlotter
from typing import Iterable
from qec_lego_bench.cli.logical_error_rate import logical_error_rate

### Define the job list

In [4]:
# cluster_node_limit
c_vec = list(range(51)) + [100 * (10 ** i) for i in range(5)]

jobs = [MonteCarloJob(c=c) for c in c_vec]


def monte_carlo_function(shots: int, c: int) -> tuple[int, LogicalErrorResult]:
    stats = logical_error_rate(decoder=f"mwpf(c={c})", code=code, noise=noise, max_shots=shots, max_errors=shots, no_progress=True, no_print=True)
    return stats.shots, LogicalErrorResult(errors=stats.errors, discards=stats.discards)

monte_carlo_function(1000, min(c_vec))

(1000, LogicalErrorResult(errors=0, discards=0))

### Define the strategy to submit jobs

In [5]:
min_shots_submitter = MinShotsSubmitter(shots=min_shots)
precision_submitter = PrecisionSubmitter(time_limit=10*3600, min_precision=0.3, target_precision=0.01)
def submitter(jobs: Iterable[MonteCarloJob]) -> list[tuple[MonteCarloJob, int]]:
    submit = min_shots_submitter(jobs)
    submit += precision_submitter(jobs)
    return submit

### Define the callback, e.g. plotting the intermediate result and the list of remaining tasks

In [6]:
from qec_lego_bench.hpc.monte_carlo import *
import matplotlib.pyplot as plt
from IPython import display
from dataclasses import field
from typing import Optional
from qec_lego_bench.hpc.plotter.logical_error_rate_plotter import *

@dataclass
class ClusterNodeLimitPlotter:
    c_vec: list[int]

    hdisplay: display.DisplayHandle = field(
        default_factory=lambda: display.display("", display_id=True)
    )
    fig: Figure = field(default_factory=lambda: plt.figure())
    def __post_init__(self):
        self.fig.clear()

    def __call__(self, executor: MonteCarloJobExecutor):
        x_vec = []
        y_vec = []
        err_vec = []
        display_c_of = lambda c: c if c != 0 else 0.5
        for c in self.c_vec:
            job = executor.get_job(c=c)
            if job is None or job.result is None:
                continue
            x_vec.append(display_c_of(c))
            stats = job.result.stats_of(job)  # type: ignore
            y_vec.append(stats.failure_rate_value)
            err_vec.append(stats.failure_rate_uncertainty)
        if len(x_vec) == 0:
            return
        fig = self.fig
        ax = fig.gca()
        ax.clear()
        ax.errorbar(x_vec, y_vec, err_vec, label=f"noise={noise}")
        ax.set_xlabel("cluster node limit")
        ax.set_ylabel("logical error rate $p_L$")
        ax.set_xlim(display_c_of(min(self.c_vec)) / 2, max(self.c_vec) * 2)
        ax.set_xscale("log")
        ax.set_ylim(1e-5, 1e-4)
        ax.set_yscale("log")
        fig.legend()
        self.hdisplay.update(fig)

In [7]:
plotter = ClusterNodeLimitPlotter(c_vec)
progress_plotter = JobProgressPlotter()
def callback(executor: MonteCarloJobExecutor):
    plotter(executor)
    progress_plotter(executor)

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

## The rest of the notebook runs the evaluation

In [8]:
try:
    from dask_jobqueue import SLURMCluster
    from dask.distributed import Client
    n_workers = slurm_maximum_jobs // slurm_cores_per_node
    assert n_workers <= 200, "Yale HPC forbids submitting more than 200 jobs per hour"
    slurm_job_folder = os.path.join(os.path.abspath(os.getcwd()), "slurm_job")
    cluster = SLURMCluster(
        queue=slurm_partition,
        cores=slurm_cores_per_node,
        processes=slurm_cores_per_node,
        memory=f"{slurm_mem_per_job * slurm_cores_per_node} GB",
        walltime=slurm_walltime,
        job_extra_directives=[f'--out="{slurm_job_folder}/%j.out"', f'--error="{slurm_job_folder}/%j.err"'],
    )
    print(cluster.job_script())
    # cluster.scale(slurm_maximum_jobs)
    cluster.adapt(minimum=slurm_maximum_jobs, maximum=slurm_maximum_jobs)  # allow respawn
    shudown_cluster = True
except Exception as e:
    print(e)
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster(n_workers=local_maximum_jobs)
    shudown_cluster = False

No module named 'dask_jobqueue'


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59180 instead
  warnings.warn(
2025-01-31 21:40:42,064 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:59204' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'monitored_job-d00f5fff5123c6fda0a76ccd883a0524', 'monitored_job-d4e133f93c0d425cd11101fd1fd362fb', 'monitored_job-e284ac6201a73674e1c7d9e115ea2ad8'} (stimulus_id='handle-worker-cleanup-1738377642.064277')
2025-01-31 21:40:42,069 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:59213' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'monitored_job-066977039287823ba5b15c4e2b1fa4b0', 'monitored_job-927c12335a9eacff93f632a4a686bb7c'} (stimulus_id='handle-worker-cleanup-1738377642.068964')
2025-01-31 21:40:

In [9]:
print(cluster.dashboard_link)
cluster

http://127.0.0.1:59180/status


LocalCluster(a7d256b1, 'tcp://127.0.0.1:59181', workers=10, threads=10, memory=32.00 GiB)

In [10]:
if shudown_cluster:
    print("will shut down the cluster after job finishes; if this is not desired, set `shudown_cluster` to False")
with Client(cluster) as client:
    try:
        executor = MonteCarloJobExecutor(
            client,
            monte_carlo_function,
            jobs,
            filename=json_filename,
        )
        executor.execute(submitter, loop_callback=callback)
    finally:
        if shudown_cluster:
            print("shutting down the whole cluster; if this is not desired, set `shudown_cluster` to False")
            client.shutdown()

2025-01-31 21:40:42,060 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('monitored_job-7e85da7efa00926a74fc25c5797f207f')" coro=<Worker.execute() done, defined at /opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/distributed/worker_state_machine.py:3606>> ended with CancelledError
2025-01-31 21:40:42,061 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('monitored_job-f5d421c45da964f3653dcd9eb8137142')" coro=<Worker.execute() done, defined at /opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/distributed/worker_state_machine.py:3606>> ended with CancelledError
2025-01-31 21:40:42,063 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('monitored_job-87f6527cb9c6e2aee08f7fb92a1cd632')" coro=<Worker.execute() done, defined at /opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/distributed/worker_

KeyboardInterrupt: 